In [1]:
import pandas as pd
import numpy as np
import torch
from scipy.spatial import distance_matrix

In [42]:

df = pd.read_csv('train.csv', header=[0, 1])
meta_df = pd.read_csv("metaData.csv")
#get target feature out of df
target = df[['Timestamp',"target_feature"]].rename(columns={"Unnamed: 1_level_1":"","Unnamed: 53_level_1":""})
target.columns = [''.join(col).strip() for col in target.columns] 
target.drop(target.iloc[0].name, inplace=True)
target["Timestamp"] = pd.to_datetime(target["Timestamp"])
df.head()

C:\Users\conny\AppData\Local\Temp\ipykernel_3148\1690104781.py:1: DtypeWarning: Columns (0,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv', header=[0, 1])


Unnamed: 0_level_0            Timestamp Wind speed (m/s)             \
             Turbine   Unnamed: 1_level_1       Kelmarsh 2 Kelmarsh 3   
0                 id                  NaN              NaN        NaN   
1                  0  2017-10-01 00:00:00         9.112875   9.197829   
2                  1  2017-10-01 00:10:00         8.319863   9.350296   
3                  2  2017-10-01 00:20:00         8.799176   8.510402   
4                  3  2017-10-01 00:30:00         8.392921   8.825861   

                                   Wind speed, Standard deviation (m/s)  \
  Kelmarsh 4 Kelmarsh 5 Kelmarsh 6                           Kelmarsh 2   
0        NaN        NaN        NaN                                  NaN   
1   8.795919   8.749384   7.684954                             1.104222   
2   8.762059   8.652318   7.421576                             1.207655   
3   9.533566   8.166586   7.797040                             1.124029   
4   8.990985   7.877046   7.750090                             1.219647   

                         ... Generator RPM (RPM)                            \
  Kelmarsh 3 Kelmarsh 4  ...          Kelmarsh 4   Kelmarsh 5   Kelmarsh 6   
0        NaN        NaN  ...                 NaN          NaN          NaN   
1   1.096619   0.999588  ...         1768.215088  1733.296875  1603.749390   
2   0.916610   0.956899  ...         1770.806885  1737.218262  1572.769409   
3   0.992841   1.343180  ...         1766.002319  1698.699219  1655.049316   
4   1.218454   0.970566  ...         1783.569824  1661.264526  1678.571777   

  Blade angle (pitch position) A (°)                                   \
                          Kelmarsh 2 Kelmarsh 3 Kelmarsh 4 Kelmarsh 5   
0                                NaN        NaN        NaN        NaN   
1                           0.183487   0.135490   0.001053     0.0000   
2                           0.051998   0.086999   0.095998     0.0605   
3                           0.168988   0.056997   0.087998     0.0090   
4                           0.084492   0.133995   0.017500     0.0000   

                        training      target_feature  
  Kelmarsh 6 Unnamed: 52_level_1 Unnamed: 53_level_1  
0        NaN                 NaN                 NaN  
1      0.000                True            8.602844  
2      0.000                True            8.125226  
3      0.016                True            7.551986  
4      0.000                True            7.912114  

[5 rows x 54 columns]

In [43]:
df.columns = ['_'.join(col).strip() for col in df.columns]  # Flatten multi-index
df.rename(columns={"Timestamp_Unnamed: 1_level_1": "Timestamp"}, inplace=True)  # Rename timestamp
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

In [44]:
df_long = df.melt(id_vars=["Timestamp"], 
                  var_name="Variable_Turbine", 
                  value_name="Value")

# Split 'Variable_Turbine' into 'Variable' and 'Turbine'
df_long[["Variable", "Turbine"]] = df_long["Variable_Turbine"].str.rsplit("_", n=1, expand=True)
df_long.drop(columns=["Variable_Turbine"], inplace=True)
df_long=df_long[df_long["Turbine"].str.contains("Kelmarsh")]
df_long.head()  # View the reshaped dataframedf

,Timestamp,Value,Variable,Turbine
130609,NaT,NaN,Wind speed (m/s),Kelmarsh 2
130610,2017-10-01 00:00:00,9.112875,Wind speed (m/s),Kelmarsh 2
130611,2017-10-01 00:10:00,8.319863,Wind speed (m/s),Kelmarsh 2
130612,2017-10-01 00:20:00,8.799176,Wind speed (m/s),Kelmarsh 2
130613,2017-10-01 00:30:00,8.392921,Wind speed (m/s),Kelmarsh 2


In [45]:
df_wide = df_long.pivot(index=['Timestamp', 'Turbine'], columns='Variable', values='Value').reset_index()
df_wide = df_wide[df_wide["Timestamp"].notna()]

In [46]:
meta_df["total_hub_height"] = meta_df["Hub Height (m)"]  + meta_df["Elevation (m)"] 

In [47]:
df_wide_meta = df_wide.merge(meta_df, left_on="Turbine",right_on="Title")

In [53]:
df_final = df_wide_meta.merge(target, on="Timestamp")

In [ ]:
coords = meta_df[['Latitude', 'Longitude', 'total_hub_height']].values  # take location vars from meta_data

# Compute pairwise distances
distances = distance_matrix(coords, coords)  # Shape: (num_turbines, num_turbines)

# Define edges: Connect turbines within a threshold distance
threshold = 5000  # Example: 5000 meters
edges = np.argwhere(distances < threshold)

# Remove self-loops
edges = edges[edges[:, 0] != edges[:, 1]]

# Convert to PyG format
edge_index = torch.tensor(edges.T, dtype=torch.long)  # Shape: (2, num_edges)

In [50]:
import torch
import pandas as pd
from torch_geometric.data import Data, Dataset
from sklearn.preprocessing import StandardScaler
import networkx as nx
from itertools import combinations

class WindTurbineGraphDataset(Dataset):
    def __init__(self, df, edge_threshold=0.5, transform=None):
        super().__init__(transform)
        self.df = df
        self.scaler = StandardScaler()
        self.edge_threshold = edge_threshold  # Define how nodes connect

        # Extract unique timestamps
        self.timestamps = df["Timestamp"].unique()
        
        # Standardize features
        feature_cols = df.columns.difference(["Timestamp", "Turbine", "target_feature"])
        self.df[feature_cols] = self.scaler.fit_transform(self.df[feature_cols])

    def len(self):
        return len(self.timestamps)

    def get(self, idx):
        """Convert a single timestamp into a graph"""
        timestamp = self.timestamps[idx]
        df_t = self.df[self.df["Timestamp"] == timestamp]

        # Extract node features
        x = torch.tensor(df_t.drop(columns=["Timestamp", "Turbine", "target_feature"]).values, dtype=torch.float)

        # Create edges (fully connected or distance-based)
        num_nodes = len(df_t)
        edge_index = torch.tensor(list(combinations(range(num_nodes), 2)), dtype=torch.long).T  # Fully connected

        # Extract target (graph-level label)
        y = torch.tensor([df_t["target_feature"].mean()], dtype=torch.float)  # Example: mean wind speed

        return Data(x=x, edge_index=edge_index, y=y)

In [51]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class WindGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # Pool over the graph
        x = self.fc(x)
        return x

In [52]:
from torch_geometric.loader import DataLoader
import torch.optim as optim

# Prepare dataset
dataset = WindTurbineGraphDataset(df)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = WindGNN(in_channels=dataset[0].x.shape[1], hidden_channels=64, out_channels=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.mse_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

ValueError: could not convert string to float: 'id'

In [ ]:
model.eval()
with torch.no_grad():
    for data in train_loader:
        data = data.to(device)
        predictions = model(data)
        print(predictions)  # Graph-level predictions